In [52]:
import itertools as it
import pandas as pd
import requests as rq

In [53]:
# Parameters
users = None
output = "recommendations.json"
host = "host.docker.internal"
port = "8070"
sample_size = 10
limits = [10]
walks = [1000]
walk_length = [100]

In [55]:
# Import users
df = pd.read_json(users)

# Compute cross product of all configuration values
configurations = list(it.product(walks, walk_length, limits))

In [57]:
def get_sample(df, offset, sample_size):
    return df.iloc[offset:offset + sample_size]

def get_recommendations(sample_df, walks, walk_length, limit):
    rows = []

    for i, row in sample_df.iterrows():
        user = row["user"]
        url = f"http://{host}:{port}/recommendation/salsa/{user}/?walks={walks}&walk_length={walk_length}&limit={limit}"
        response = rq.get(url)

        status_code = response.status_code
        duration = response.elapsed.total_seconds()
        recommendations = response.json() if response.status_code == 200 else []
        rows.append((walks, walk_length, limit, url, status_code, duration, user, recommendations))
    
    return pd.DataFrame(rows, columns=["walks", "walk_length", "limit", "url", "status_code", "duration", "user", "recommendations"])

In [ ]:
# Perform API requests
offset = 0
result_dfs = []

for (walks, walk_length, limit) in configurations:
    print(f"Fetch recommendations: walks {walks}, walk_length {walk_length}, limit {limit}")
    sample_df = get_sample(df, offset, sample_size)
    result_dfs.append(get_recommendations(sample_df, walks, walk_length, limit))
    
result_df = pd.concat(result_dfs)

In [58]:
# Save results
result_df.to_json(output, index=False, orient="table")